In [7]:
import codecs
import re
import pandas as pd
import requests
import json
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
URL = "https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:1/Y:1/station:0"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'Accept': 'application/json',
    'Referer': 'https://www.marinetraffic.com/',
}
try:
    response = requests.get(URL,headers=headers)
    response.raise_for_status
    print("Status Code:", response.status_code)
    print("Response Text:", response.text[:500])

    data = response.json()

except requests.exceptions.RequestException as e:
    print("Abort THE MISSION CHIGGA",e)

Status Code: 403
Response Text: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" 
Abort THE MISSION CHIGGA Expecting value: line 1 column 1 (char 0)


In [8]:

# Set up Selenium with Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless to stay stealth
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36')

# Auto-install ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # Step 1: Load main page to grab cookies and session
    print("Hitting main page to snag cookies...")
    driver.get("https://www.marinetraffic.com/en/ais/home")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located(("tag name", "body")))  # Wait for page load
    time.sleep(3)  # Extra buffer for JS/cookies

    # Grab cookies
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    print("Cookies grabbed:", cookies)

    # Step 2: Try to find Vessel-Token or other headers (if needed)
    # Check DevTools > Application > Local Storage or run JS to find tokens
    try:
        vessel_token = driver.execute_script("return localStorage.getItem('Vessel-Token') || ''")
        print("Vessel-Token:", vessel_token)
    except:
        vessel_token = None
        print("No Vessel-Token found in localStorage")

    # Headers for XHR request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
        'Accept': 'application/json',
        'Referer': 'https://www.marinetraffic.com/',
        'X-Requested-With': 'XMLHttpRequest',
    }
    if vessel_token:
        headers['Vessel-Token'] = vessel_token  # Add if found

    # Step 3: Hit the XHR endpoint for a single tile
    base_url = "https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:{}/Y:{}/station:0"
    x, y = 1, 1  # Start with one tile
    url = base_url.format(x, y)

    print(f"Trying XHR URL: {url}")
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        print("Response Text (first 500 chars):", response.text[:500])

        # Parse JSON
        data = response.json()
        ships = data.get('data', {}).get('rows', [])
        print(f"Got {len(ships)} ships from X:{x}, Y:{y}")

        # Convert to DataFrame
        df = pd.DataFrame(ships)
        print("DataFrame head:", df.head().to_string())

        # Save to CSV
        df.to_csv('ships_X1_Y1.csv', index=False, encoding='utf-8')
        print("Saved to ships_X1_Y1.csv")

    except requests.exceptions.RequestException as e:
        print("XHR failed, damn it:", e)
        print("Response Text (if any):", response.text[:500] if 'response' in locals() else "No response")
        raise  # Let's debug this first

    # Step 4: Loop through multiple tiles (uncomment once single tile works)
    """
    all_ships = []
    x_range = range(0, 5)  # Adjust based on zoom/area
    y_range = range(0, 5)
    for x in x_range:
        for y in y_range:
            url = base_url.format(x, y)
            try:
                response = requests.get(url, headers=headers, cookies=cookies)
                response.raise_for_status()
                data = response.json()
                ships = data.get('data', {}).get('rows', [])
                all_ships.extend(ships)
                print(f"Got {len(ships)} ships from X:{x}, Y:{y}")
                time.sleep(2)  # Avoid banhammer
            except requests.exceptions.RequestException as e:
                print(f"Failed for X:{x}, Y:{y}: {e}")
                continue

    # Save all ships
    df_all = pd.DataFrame(all_ships)
    df_all.to_csv('all_ships.csv', index=False, encoding='utf-8')
    print(f"Total ships collected: {len(all_ships)}, saved to all_ships.csv")
    """

except Exception as e:
    print("Mission blown to shit:", e)

finally:
    driver.quit()  # Clean up


Hitting main page to snag cookies...
Cookies grabbed: {'cf_clearance': 'aXUFlTUz74h8lpW0zPe8a3QBy.BfA0zj_8tKBlCPpJE-1758285392-1.2.1.1-S2yZ9xI180qr1bpdpyJ01oybx0VdORpl16cAmk2Dankt_TdGGNWaA__fRPOvF.sCEGp9ghrX4iOmXQMqwK_g3MVTAA4I8ffjrobZ6tSAg3tXNWb4UuSseVQJRadAbP8.pwdKMB_0nKrKYO_r1gPe0TSqRLZPO1EYPzptvlP6ydM2Scwu4hlvxR7Lww5tr236IfoDjOF.8J57rBgLJnrbnly2h1.kA4xw3MyfQrLEhA8', 'usprivacy': '1N--', '_cfuvid': '9OKzW3pE9mNYN0Zu.yPlL7nWtWt9Pgj0yxX4VtyoU7Y-1758285389957-0.0.1.1-604800000', '__cf_bm': '2O9_bWaCZPFAqJzwKrutPy3gIjEXoxWxHoNLBw80Afo-1758285389-1.0.1.1-1BoO3jPLibKGNkFSRFiccLddOXwqxugVtEyzWt.Y4NDO9MkgTRIWCxLW1XcSPyf9.u76udzUJFnFmzk8nkXpEwUs4wCGLkZNdiQ01HK5hDE'}
Vessel-Token: 
Trying XHR URL: https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:1/Y:1/station:0
XHR failed, damn it: 403 Client Error: Forbidden for url: https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:1/Y:1/station:0
Response Text (if any): <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" la